# The purpose of this script is to extract the GTD database of incidents.

There are 2 parts ot this script. The first part extracts a list of GTD_ID from the search results.
The second part extracts a dataframe of GTD incidents with the list of GTD_ID.

This portion scrapes the GTD search results for all the GTD ID. The IDs will be used to extract profile information from GTD
we will be scraping between 2015-2017

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
'''scraping the table in the iframe in GTD'''

url = "https://www.start.umd.edu/gtd/search/Results.aspx?start_yearonly=&end_yearonly=&start_year=2015&start_month=1&start_day=1&end_year=2017&end_month=12&end_day=31&region=5&asmSelect0=&asmSelect1=&criterion1=yes&criterion2=yes&criterion3=yes&dtp2=all&success=yes&casualties_type=b&casualties_max="

driver = webdriver.Firefox()

driver.get(url)
driver.maximize_window()

In [3]:
#function to select next page
def next_pg():
    try:
        driver.implicitly_wait(20)
        next_pg = driver.find_element_by_css_selector(".search-bottom-nav > div:nth-child(2) > a:nth-child(2)")
        next_pg.click()
        
    except:
        print('no more',end='\r')

#function to scrape links from the site and return the id number
def scrape_links():
    '''scrape table in site and return id number associated with the link'''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tables  = soup.find_all('div', class_='advanced-search-results')

    data = []
    
    for i in tables:
        a = i.find_all('tr')
        for b in a:
            b2 = b.find_all('a', href=True) #print(type(b2)) #Class == ResultSet
            b2 = [link.string for link in b2]
            data.append(b2)
            #print(data)
    return data

Need to manually input the number of pages. #future iterations can be made more robust by using a while loop

In [4]:
pages = 121 #checked

In [5]:
main_df = pd.DataFrame()

for i in range(pages):
    lnk = scrape_links()
    df2 = pd.DataFrame(lnk)
    df2.columns = df2.iloc[0]
    
    #df = pd.DataFrame(scrape_table())
    #df3 = pd.concat([df,df2],axis=1)
    #main_df.append(df)
    #main_df = pd.concat([main_df,df3],axis=0)
    main_df = pd.concat([main_df,df2])
    
    next_pg()
    print('going to next '+ str(i) ,end='\r')
print('completed')

completednext 120


In [6]:
main_df

,GTD ID,COUNTRY,CITY,FATALITIES,INJURED
0,GTD ID,COUNTRY,CITY,FATALITIES,INJURED
1,201712310032,None,None,None,None
2,201712310030,None,None,None,None
3,201712310016,None,None,None,None
4,201712310011,None,None,None,None
5,201712310010,None,None,None,None
6,201712300010,None,None,None,None
7,201712290008,None,None,None,None
8,201712280019,None,None,None,None
9,201712270030,None,None,None,None


In [8]:
driver.quit()
main_df.reset_index().drop_duplicates()

gtd_df = main_df.drop_duplicates(keep='first')
gtd_df = gtd_df['GTD ID']
gtd_df.reset_index(drop=True, inplace=True)
gtd_df.to_csv('GTD_ID_SEA_2015-2017.csv', encoding='utf-8')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  import sys


Exported the data into a csv 'GTD_ID_SEA_2015-2017.csv'

This is the second portion which extracts a dataframe of GTD incidents with the list of GTD_ID.

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import json
from pandas.io.json import json_normalize

import pandas.io.common

In [10]:
df = pd.read_csv('GTD_ID_SEA_2015-2017.csv')
lst = df['GTD ID']

In [11]:
#this script is for parsing the site on START

#getting the url
def request_start(url):
    #try not to use verify=False, you are vulnerable when you do this
    r = requests.get(url, verify =False)
    soup = BeautifulSoup(r.text,'html.parser')
    return soup

#extracting the relevant elements and creating a list of json
def extract(soup):
    '''
    print(incident_summary(soup))
    print(summary_overview(soup))
    print(source(soup))
    print(what(soup))
    print(how(soup))
    print(who(soup))
    '''
    #Entry
    entry  = [incident_summary(soup), source(soup)] + summary_overview(soup) + what(soup) + how(soup) + who(soup)
    #entry  = [incident_summary(soup), source(soup)] + summary_overview(soup)
    #print('entry type: '+ str(type(entry)))
    #print(entry)
    return entry
    
#incident summary
def incident_summary(soup):
    incident_summary = soup.find('div', {"id": "incidentSummary"})
    incident_s = json.dumps({"0_incident_summary": incident_summary.text.replace('\n',' ')})
    return incident_s
    #print(incident_s)
    #print('incident s '+ str(type(incident_s)))
    
##summary overview
def summary_overview(soup):
    summary_overview = soup.find('div', {"class": "summary-overview"})
    leftHead = summary_overview.find_all('span', {"class": "leftHead"})
    LH = [i.text for i in leftHead]
    leftLarge = summary_overview.find_all('span', {"class": "leftLarge"})
    LL = [i.text for i in leftLarge]

    overview = list(zip(LH,LL))
    Summ_Over = []
    for i in overview:
        a = json.dumps({i[0]:i[1]})
        Summ_Over.append(a)
    return Summ_Over

#sources
def source(soup):
    incident_sources = soup.find('div', {"id": "sources"})
    sources = incident_sources.find_all('table')
    df = pd.read_html(str(sources))
    #print(df[0].values.tolist())
    if df:
        src = json.dumps({"1_src":df[0].values.tolist()})
    else:
        src = json.dumps({"1_src":'NA'})
    #src = mk_table(sources)
    #print(src)
    #print('src '+ str(type(src)))
    return src

#what
def what(soup):
    i_w = soup.find('div', {"id": "what"})
    wut = []
    
    results_list = i_w.find_all('table')
    for i in results_list:
        c = i.find('caption').get_text()
        #print(c)
        a = str(i)
        b = pd.read_html(a)
        
        for i in b:
            d = i.values.tolist()
            e = json.dumps({c:d})
            wut.append(e)
    
    #print(wut)
    return wut

#how
def how(soup):
    i_h = soup.find('div', {"id": "how"})
    #print('type of i_h '+str(type(i_h)))
    #results = i_h.find_all('div', {"class": "results"})
    howe = []
    results_list = i_h.find_all('table')    
    for i in results_list:
        c = i.find('caption').get_text()
        #print(c)
        a = str(i)
        b = pd.read_html(a)
        
        for i in b:
            d = i.values.tolist()
            e = json.dumps({c:d})
            howe.append(e)
    
    return howe
    
#who
def who(soup):
    i_who = soup.find('div', {"id": "who"})
    whu = []
    
    results_list = i_who.find_all('table')
    for i in results_list:
        c = i.find('caption').get_text()
        #print(c)
        a = str(i)
        b = pd.read_html(a)
        
        
        for i in b:
            d = i.values.tolist()
            e = json.dumps({c:d})
            whu.append(e)
    return whu

#this function fetches gtd dataframe based on the id
def fetchGTD(id):
    url = 'https://www.start.umd.edu/gtd/search/IncidentSummary.aspx?gtdid={0}'.format(id)
    a = request_start(url)
    print('getting '+ url)
        
    datarow = extract(a)
    #print(datarow)

    data = {}
    for i in datarow:
        #print(i)
        a = json.loads(i)
        #print(type(a))
        #print(a)
        data.update(a)
    #print(data)
        
    #results = pd.DataFrame.from_records(data)
    
    results = pd.DataFrame.from_dict(data, orient='index')
    results = results.transpose()
    
    #print(results)
    
    return results

In [12]:
'''new execution'''
count=1
error_id = []

for i in lst:
    print(str(round(count/len(lst)*100,2)) +'% complete')
    count +=1
    
    entry = fetchGTD(i)
    #print(type(a))
    
    try:                       
        with open('GTD_case_Details_2015_2017.csv', 'a', encoding="utf-8") as f:
            entry.to_csv(f, mode='a')
   
    except pandas.io.common.EmptyDataError:
        entry.to_csv('GTD_case_Details_2015_2017.csv')
    
    except Exception as e:
        '''throw the error id into a list for error handling later'''
        error_id.append(i)
        print('error detected. putting error in error log')
        
        error_msg = [error_id,e]
        
        with open('GTD_case_Details_2015_2017.csv', 'a', encoding="utf-8") as f:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(error_msg)


0.04% complete


C:\Users\User\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


getting https://www.start.umd.edu/gtd/search/IncidentSummary.aspx?gtdid=201712310032
0.08% complete


C:\Users\User\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


getting https://www.start.umd.edu/gtd/search/IncidentSummary.aspx?gtdid=201712310030
0.12% complete


...

getting https://www.start.umd.edu/gtd/search/IncidentSummary.aspx?gtdid=201501050056
99.96% complete


C:\Users\User\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


getting https://www.start.umd.edu/gtd/search/IncidentSummary.aspx?gtdid=201501020080
100.0% complete


C:\Users\User\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


getting https://www.start.umd.edu/gtd/search/IncidentSummary.aspx?gtdid=201501010014


cleaning and flattening the dataframe

In [24]:
'''dealing with malformed csv'''
import csv
import pandas as pd
from IPython.display import display


d_lst = []
with open('GTD_case_Details_2015_2017.csv',encoding="utf8") as f:
    reader = csv.reader(f)    
    for row in reader:
        if row:
            d_lst.append(row)
        
len(d_lst)
display((0,d_lst[0]))
display((1,d_lst[1]))
display((2,d_lst[2]))
display((3,d_lst[3]))
display((4,d_lst[4]))
display((5,d_lst[5]))

(0,
 ['',
  '0_incident_summary',
  '1_src',
  'GTD ID:',
  'When:',
  'Country:',
  'Region:',
  'Province/administrativeregion/u.s. state:',
  'City:',
  'Attack Information',
  'Target Information (more)',
  'Additional Information',
  'Weapon Information',
  'Perpetrator Group Information',
  'Perpetrator Statistics',
  'Casualty Information'])

(1,
 ['0',
  '  Incident Summary: \t\t12/31/2017: An explosive device was discovered and defused at a plaza in Cotabato City, Maguindanao, Philippines. No group claimed responsibility for the incident. \t ',
  '[[\'"Security tightened in Cotabato following IED discovery," Tempo, January 4, 2018."\'], [\'"Security tightened in Cotabato City," Manila Bulletin, January 3, 2018."\']]',
  '201712310032',
  '2017-12-31',
  'Philippines',
  'Southeast Asia',
  'Maguindanao',
  'Cotabato City',
  "[['Type of Attack (more)', 'Bombing/Explosion'], ['Successful Attack? (more)', 'No']]",
  "[['Name of Entity', 'Unknown', nan, nan], ['Specific Description', 'Unknown', nan, nan], ['Nationality of Target', 'Philippines', nan, nan]]",
  "[['Suicide Attack?', 'No'], ['Part of Multiple Incident?', 'No'], ['Criterion 1 (more)', 'Yes'], ['Criterion 2 (more)', 'Yes'], ['Criterion 3 (more)', 'Yes'], ['Doubt Terrorism Proper (more)', 'No']]",
  "[['Explosives/Bombs/Dynamite', 'Unknown Explosive Type'], ['Wea

(2,
 ['',
  '0_incident_summary',
  '1_src',
  'GTD ID:',
  'When:',
  'Country:',
  'Region:',
  'Province/administrativeregion/u.s. state:',
  'City:',
  'Location Details:',
  'Attack Information',
  'Target Information (more)',
  'Additional Information',
  'Weapon Information',
  'Perpetrator Group Information',
  'Perpetrator Statistics',
  'Casualty Information'])

(3,
 ['0',
  '  Incident Summary: \t\t12/31/2017: Assailants set fire to houses in Kubentog, Datu Hoffer, Maguindanao, Philippines. There were no reported casualties in the attack. No group claimed responsibility for the incident; however, sources attributed the attack to the Bangsamoro Islamic Freedom Movement (BIFM). \t ',
  '[[\'"Maguindanao clashes trap tribe members," Philippines Daily Inquirer, January 3, 2018."\']]',
  '201712310030',
  '2017-12-31',
  'Philippines',
  'Southeast Asia',
  'Maguindanao',
  'Kubentog',
  'The incident occurred in the Datu Hoffer district.',
  "[['Type of Attack (more)', 'Facility/Infrastructure Attack'], ['Successful Attack? (more)', 'Yes']]",
  "[['Name of Entity', 'Not Applicable', nan, nan], ['Specific Description', 'Houses', nan, nan], ['Nationality of Target', 'Philippines', nan, nan]]",
  "[['Suicide Attack?', 'No'], ['Part of Multiple Incident?', 'No'], ['Criterion 1 (more)', 'Yes'], ['Criterion 2 (more)', 'Yes'], ['Criterion 3 (more)', 'Ye

(4,
 ['',
  '0_incident_summary',
  '1_src',
  'GTD ID:',
  'When:',
  'Country:',
  'Region:',
  'Province/administrativeregion/u.s. state:',
  'City:',
  'Location Details:',
  'Attack Information',
  'Target Information (more)',
  'Additional Information',
  'Weapon Information',
  'Perpetrator Group Information',
  'Perpetrator Statistics',
  'Casualty Information'])

(5,
 ['0',
  '  Incident Summary: \t\t12/31/2017: A roadside bomb detonated targeting a police vehicle in Poblacion Mother, Tuayan, Shariff Aguak, Maguindanao, Philippines. A police officer was killed and five others were injured in the blast. No group claimed responsibility for the incident; however, sources attributed the attack to the Bangsamoro Islamic Freedom Movement (BIFM). \t ',
  '[[\'"Cop, 2 others killed in bomb blasts in Mindanao," Philippines Daily Inquirer, January 1, 2018."\'], [\'"Cop killed, 7 injured in Maguindanao IED blasts," Philippine Star, January 1, 2018."\'], [\'"3 dead, scores injured in Mindanao blasts," Manila Times (Philippines), January 2, 2018."\']]',
  '201712310016',
  '2017-12-31',
  'Philippines',
  'Southeast Asia',
  'Maguindanao',
  'Shariff Aguak',
  'The incident occurred in the Poblacion Mother area of the Tuayan neighborhood.',
  "[['Type of Attack (more)', 'Bombing/Explosion'], ['Successful Attack? (more)', 'Yes']]",
  "[['Name of Entity', 'Ph

In [25]:
m_df = pd.DataFrame()
a, b = 0, 1
while b < len(d_lst):
    l_df = pd.DataFrame([d_lst[a], d_lst[b]])
    l_df.rename(columns=l_df.iloc[0],inplace=True)
    l_df.reset_index(drop=True, inplace=True)
    l_df.drop(l_df.index[[0]], inplace=True)
    if m_df.empty == False:
        m_df = pd.concat([m_df,l_df], ignore_index=True)
    else:
        m_df = l_df
    a += 2
    b += 2
m_df

'''case details created'''
m2_df = m_df.drop('',axis=1)
m2_df.to_csv('1_GTD_case_details_2015_2017.csv') # for storage purposes

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [26]:
m2_df

,0_incident_summary,1_src,Additional Information,Attack Information,Casualty Information,City:,Country:,GTD ID:,Location Details:,Perpetrator Group Information,Perpetrator Statistics,Province/administrativeregion/u.s. state:,Region:,Target Information (more),Weapon Information,When:
0,Incident Summary: \t\t12/31/2017: An explosi...,"[['""Security tightened in Cotabato following I...","[['Suicide Attack?', 'No'], ['Part of Multiple...","[['Type of Attack (more)', 'Bombing/Explosion'...","[['Total Number of Casualties', '0 Fatalities ...",Cotabato City,Philippines,201712310032,NaN,"[['Unknown', 'No']]","[['Number of Perpetrators', 'Unknown'], ['Numb...",Maguindanao,Southeast Asia,"[['Name of Entity', 'Unknown', nan, nan], ['Sp...","[['Explosives/Bombs/Dynamite', 'Unknown Explos...",2017-12-31
1,Incident Summary: \t\t12/31/2017: Assailants...,"[['""Maguindanao clashes trap tribe members,"" P...","[['Suicide Attack?', 'No'], ['Part of Multiple...","[['Type of Attack (more)', 'Facility/Infrastru...","[['Total Number of Casualties', '0 Fatalities ...",Kubentog,Philippines,201712310030,The incident occurred in the Datu Hoffer distr...,[['Bangsamoro Islamic Freedom Movement (BIFM)'...,"[['Number of Perpetrators', 'Unknown'], ['Numb...",Maguindanao,Southeast Asia,"[['Name of Entity', 'Not Applicable', nan, nan...","[['Incendiary', 'Arson/Fire']]",2017-12-31
2,Incident Summary: \t\t12/31/2017: A roadside...,"[['""Cop, 2 others killed in bomb blasts in Min...","[['Suicide Attack?', 'No'], ['Part of Multiple...","[['Type of Attack (more)', 'Bombing/Explosion'...","[['Total Number of Casualties', '1 Fatalities ...",Shariff Aguak,Philippines,201712310016,The incident occurred in the Poblacion Mother ...,[['Bangsamoro Islamic Freedom Movement (BIFM)'...,"[['Number of Perpetrators', 'Unknown'], ['Numb...",Maguindanao,Southeast Asia,"[['Name of Entity', 'Philippine National Polic...","[['Explosives/Bombs/Dynamite', 'Unknown Explos...",2017-12-31
3,Incident Summary: \t\t12/30/2017: Assailants...,"[['""Cops hunt North Cotabato bombers,"" Philipp...","[['Suicide Attack?', 'No'], ['Part of Multiple...","[['Type of Attack (more)', 'Bombing/Explosion'...","[['Total Number of Casualties', '1 Fatalities ...",Pikit district,Philippines,201712310011,NaN,"[['Unknown', 'No']]","[['Number of Perpetrators', 'Unknown'], ['Numb...",North Cotabato,Southeast Asia,"[['Name of Entity', 'Pacalna Variety Store', n...","[['Explosives/Bombs/Dynamite', 'Grenade'], ['W...",2017-12-30
4,Incident Summary: \t\t12/31/2017: Assailants...,"[['""3 slain in Maguindanao roadside bombings,""...","[['Suicide Attack?', 'No'], ['Part of Multiple...","[['Type of Attack (more)', 'Facility/Infrastru...","[['Total Number of Casualties', '0 Fatalities ...",Limpogo,Philippines,201712310010,The incident occurred in the Datu Hoffer Ampat...,[['Bangsamoro Islamic Freedom Movement (BIFM)'...,"[['Number of Perpetrators', 'Unknown'], ['Numb...",Maguindanao,Southeast Asia,"[['Name of Entity', 'Datu Hoffer Ampatuan Town...","[['Firearms', 'Unknown Gun Type'], ['Incendiar...",2017-12-31
5,Incident Summary: \t\t12/30/2017: Assailants...,"[['""Cops hunt North Cotabato bombers,"" Philipp...","[['Suicide Attack?', 'No'], ['Part of Multiple...","[['Type of Attack (more)', 'Bombing/Explosion'...","[['Total Number of Casualties', '0 Fatalities ...",Cotabato City,Philippines,201712300010,The incident occurred in the Poblacion neighbo...,"[['Unknown', 'No']]","[['Number of Perpetrators', 'Unknown'], ['Numb...",Maguindanao,Southeast Asia,"[['Name of Entity', 'Not Applicable', nan, nan...","[['Explosives/Bombs/Dynamite', 'Grenade'], ['W...",2017-12-30
6,Incident Summary: \t\t12/28/2017: Assailants...,"[['""NPA abducts police officer in Cotabato,"" M...","[['Suicide Attack?', 'No'], ['Part of Multiple...","[['Type of Attack (more)', 'Hostage Taking (Ki...","[['Total Number of Casualties', 'Unknown'], ['...",Near Tuael,Philippines,201712290008,The incident occurred in the President Roxas d...,"[[""New People'

In [27]:
'''flatten those columns '''
import ast
import re
from IPython.display import display

def convert_column(df, column_name):
    a = []
    for i in df[column_name]:
        try:
            b = ast.literal_eval(i)
            a.append(b)
        except ValueError:
            #print('Value Error')
            b = re.findall("\[(.*?)\]", i)
            a.append(b)
        except IndentationError:
            regex = re.compile(r'[\n\r\t]')
            b = regex.sub(" ", i)
            a.append(b)
    #print(a)
    try:
        n_df = pd.DataFrame(a)
    except:
        n_df = pd.Series(a).to_frame()
        
    n_df.columns = [column_name + '_' + str(col) for col in n_df.columns]
    
    return n_df

def flat2(df,col_name):
    a = pd.DataFrame() 
    for i in df[col_name]:
        b = pd.DataFrame(i).transpose()
        
        if len(b.columns) == 3:
            b.columns= [col_name, str(col_name + '_subgroup'), str(col_name + '_claim_responsibility')] #rename this to col name, col name plus string + claim responsibility
            a = pd.concat([a,b])
        elif len(b.columns) == 2:    
            b.columns= [col_name,str(col_name + '_claim_responsibility')] #rename this to col name, col name plus string + claim responsibility
            a = pd.concat([a,b])
        else:
            #print(i)
            b = pd.DataFrame([i])
            b.columns = [col_name]
            a = pd.concat([a,b])
    return a

In [30]:
df2 = pd.read_csv('1_GTD_case_details_2015_2017.csv').drop('Unnamed: 0',1)
lst = []

for i in ['0_incident_summary','1_src', 'Additional Information',
       'Attack Information', 'Casualty Information','Perpetrator Group Information',
       'Perpetrator Statistics','Target Information (more)', 'Weapon Information']:
    a = convert_column(df2,i)
    lst.append(a)
    print(str(i) + ' ok')
    
'''to clean the GTD group more'''    
main_df = pd.concat(lst,axis=1)
display(main_df)

ldf = pd.DataFrame()
for i in ['Perpetrator Group Information_0', 'Perpetrator Group Information_1',
       'Perpetrator Group Information_2']:
    a = flat2(main_df,i)
    #print(a)
    ldf = pd.concat([ldf,a],axis=1)
    print(str(i) + ' ok')

ldf.reset_index(drop=True, inplace=True)
display(ldf)

main_df = pd.concat([main_df,ldf],axis=1, join_axes=[main_df.index])
main_df2 = pd.concat([main_df,df2['City:'],df2['GTD ID:'],df2['Country:'],df2['Location Details:'],df2['Province/administrativeregion/u.s. state:'],
       df2['Region:'],df2['When:']],axis=1)

main_df2['GTD ID:']=main_df2['GTD ID:'].apply(str)
main_df2.to_csv('2_GTD_case_details_2015_2017.csv')

0_incident_summary ok
1_src ok
Additional Information ok
Attack Information ok
Casualty Information ok
Perpetrator Group Information ok
Perpetrator Statistics ok
Target Information (more) ok
Weapon Information ok


,0_incident_summary_0,1_src_0,1_src_1,1_src_2,Additional Information_0,Additional Information_1,Additional Information_2,Additional Information_3,Additional Information_4,Additional Information_5,...,Target Information (more)_6,Target Information (more)_7,Target Information (more)_8,Target Information (more)_9,Target Information (more)_10,Weapon Information_0,Weapon Information_1,Weapon Information_2,Weapon Information_3,Weapon Information_4
0,Incident Summary: 12/31/2017: An explosive...,"[""Security tightened in Cotabato following IED...","[""Security tightened in Cotabato City,"" Manila...",None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,None,None,None,None,None,"[Explosives/Bombs/Dynamite, Unknown Explosive ...","[Weapon Details, Weapon Details]",[An explosive device containing a detonating c...,None,None
1,Incident Summary: 12/31/2017: Assailants s...,"[""Maguindanao clashes trap tribe members,"" Phi...",None,None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,None,None,None,None,None,"[Incendiary, Arson/Fire]",None,None,None,None
2,Incident Summary: 12/31/2017: A roadside b...,"[""Cop, 2 others killed in bomb blasts in Minda...","[""Cop killed, 7 injured in Maguindanao IED bla...","[""3 dead, scores injured in Mindanao blasts,"" ...","[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,None,None,None,None,None,"[Explosives/Bombs/Dynamite, Unknown Explosive ...","[Weapon Details, Weapon Details]","[A roadside bomb was used in the attack., A ro...",None,None
3,Incident Summary: 12/30/2017: Assailants t...,"[""Cops hunt North Cotabato bombers,"" Philippin...","[""Philippines: Highlights of Terrorist, Counte...",None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,None,None,None,None,None,"[Explosives/Bombs/Dynamite, Grenade]","[Weapon Details, Weapon Details]","[A thrown grenade was used in the attack., A t...",None,None
4,Incident Summary: 12/31/2017: Assailants s...,"[""3 slain in Maguindanao roadside bombings,"" P...","[""Philippines: Highlights of Terrorist, Counte...","[""BIFF gunmen torch abandoned houses in Maguin...","[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,None,None,None,None,None,"[Firearms, Unknown Gun Type]","[Incendiary, Arson/Fire]",None,None,None
5,Incident Summary: 12/30/2017: Assailants t...,"[""Cops hunt North Cotabato bombers,"" Philippin...","[""Philippines: Highlights of Terrorist, Counte...",None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,None,None,None,None,None,"[Explosives/Bombs/Dynamite, Grenade]","[Weapon Details, Weapon Details]","[A thrown grenade was used in the attack., A t...",None,None
6,Incident Summary: 12/28/2017: Assailants a...,"[""NPA abducts police officer in Cotabato,"" Man...","[""Police official taken by armed men in North ...","[""Philippines: Suspected communist rebels abdu...","[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,None,None,None,None,None,"[Firearms, Unknown Gun Type]",None,None,None,None
7,Incident Summary: 12/28/2017: Assailants a...,"[""Cop slain, 5 colleagues wounded in BIFF atta...","[""BIFF kills 2 kidnapped f

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Perpetrator Group Information_0 ok


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Perpetrator Group Information_1 ok
Perpetrator Group Information_2 ok


,Perpetrator Group Information_0,Perpetrator Group Information_0_claim_responsibility,Perpetrator Group Information_0_subgroup,Perpetrator Group Information_1,Perpetrator Group Information_1_claim_responsibility,Perpetrator Group Information_2,Perpetrator Group Information_2_claim_responsibility
0,Unknown,No,NaN,None,NaN,None,NaN
1,Bangsamoro Islamic Freedom Movement (BIFM),No,NaN,None,NaN,None,NaN
2,Bangsamoro Islamic Freedom Movement (BIFM),No,NaN,None,NaN,None,NaN
3,Unknown,No,NaN,None,NaN,None,NaN
4,Bangsamoro Islamic Freedom Movement (BIFM),No,NaN,None,NaN,None,NaN
5,Unknown,No,NaN,None,NaN,None,NaN
6,New People's Army (NPA),Yes (Confirmed: Unknown; Mode: Personal claim),Mt. Apo Sub-Regional Command,None,NaN,None,NaN
7,Bangsamoro Islamic Freedom Movement (BIFM),No,NaN,None,NaN,None,NaN
8,New People's Army (NPA),No,Southern Mindanao Regional Command (SMRC),None,NaN,None,NaN
9,Unknown,No,NaN,None,NaN,None,NaN


In [31]:
main_df2

,0_incident_summary_0,1_src_0,1_src_1,1_src_2,Additional Information_0,Additional Information_1,Additional Information_2,Additional Information_3,Additional Information_4,Additional Information_5,...,Perpetrator Group Information_1_claim_responsibility,Perpetrator Group Information_2,Perpetrator Group Information_2_claim_responsibility,City:,GTD ID:,Country:,Location Details:,Province/administrativeregion/u.s. state:,Region:,When:
0,Incident Summary: 12/31/2017: An explosive...,"[""Security tightened in Cotabato following IED...","[""Security tightened in Cotabato City,"" Manila...",None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,NaN,None,NaN,Cotabato City,201712310032,Philippines,NaN,Maguindanao,Southeast Asia,2017-12-31
1,Incident Summary: 12/31/2017: Assailants s...,"[""Maguindanao clashes trap tribe members,"" Phi...",None,None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,NaN,None,NaN,Kubentog,201712310030,Philippines,The incident occurred in the Datu Hoffer distr...,Maguindanao,Southeast Asia,2017-12-31
2,Incident Summary: 12/31/2017: A roadside b...,"[""Cop, 2 others killed in bomb blasts in Minda...","[""Cop killed, 7 injured in Maguindanao IED bla...","[""3 dead, scores injured in Mindanao blasts,"" ...","[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,NaN,None,NaN,Shariff Aguak,201712310016,Philippines,The incident occurred in the Poblacion Mother ...,Maguindanao,Southeast Asia,2017-12-31
3,Incident Summary: 12/30/2017: Assailants t...,"[""Cops hunt North Cotabato bombers,"" Philippin...","[""Philippines: Highlights of Terrorist, Counte...",None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,NaN,None,NaN,Pikit district,201712310011,Philippines,NaN,North Cotabato,Southeast Asia,2017-12-30
4,Incident Summary: 12/31/2017: Assailants s...,"[""3 slain in Maguindanao roadside bombings,"" P...","[""Philippines: Highlights of Terrorist, Counte...","[""BIFF gunmen torch abandoned houses in Maguin...","[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,NaN,None,NaN,Limpogo,201712310010,Philippines,The incident occurred in the Datu Hoffer Ampat...,Maguindanao,Southeast Asia,2017-12-31
5,Incident Summary: 12/30/2017: Assailants t...,"[""Cops hunt North Cotabato bombers,"" Philippin...","[""Philippines: Highlights of Terrorist, Counte...",None,"[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,NaN,None,NaN,Cotabato City,201712300010,Philippines,The incident occurred in the Poblacion neighbo...,Maguindanao,Southeast Asia,2017-12-30
6,Incident Summary: 12/28/2017: Assailants a...,"[""NPA abducts police officer in Cotabato,"" Man...","[""Police official taken by armed men in North ...","[""Philippines: Suspected communist rebels abdu...","[Suicide Attack?, No]","[Part of Multiple Incident?, No]","[Criterion 1 (more), Yes]","[Criterion 2 (more), Yes]","[Criterion 3 (more), Yes]","[Doubt Terrorism Proper (more), No]",...,NaN,None,NaN,Near Tuael,201712290008,Philippines,The incident occurred in the President Roxas d...,North Cotabato,Southeast Asia,2017-12-28
7,Incident Summary: 12/28/2017: Assailants a...,"[""Cop slain, 5 colleagues wounded in BIFF atta...","[""BIFF kills 2 kidnapped farmers, burns villag...","[""Philippines: Hi